## Films to show in the sing in web page

On the sign-in page of the application, users will be required to rate at least 10 movies. To facilitate this, I will retrieve a list of 80 popular movies, as well as an additional 20 movies of my own selection. Users will be able to browse this list, select a movie, and provide a rating.

Since the movie titles in the original dataset are in English, but my application targets users in Mexico, I will need to fetch the movie titles in Spanish to better serve the local audience.

In [1]:
import pandas as pd
import mysql.connector
import requests
import os
from dotenv import load_dotenv

# Load the environment variables .env
load_dotenv()

True

Previously, we filtered and selected several movies from the original dataset. Now, we will use the cleaned dataset to select the top 80 films.

In [2]:
df = pd.read_csv('../Data/Processed/movies_final.csv')
df.head()

,movieId,id,title,genres,overview,release_date,popularity,runtime,status,tagline,vote_average,vote_count,poster_path,backdrop_path,cast,director,keywords
0,221850,614696,#Alive,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...","As a grisly virus rampages a city, a lone man ...",2020-06-24,3.6641,98.0,Released,You must survive.,7.231,1868.0,/lZPvLUMYEPLTE2df1VW5FHTYC8N.jpg,/k2SY15W9QXH9qL8f4a4BbytV1BE.jpg,"['Yoo Ah-in', 'Park Shin-hye', 'Lee Hyun-wook']",Cho Il,"['escape', 'alone', 'survival', 'drone', 'zomb..."
1,212989,605734,#Iamhere,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",Stéphane lives a quiet life as an eminent Fren...,2020-02-05,0.6237,100.0,Released,NaN,5.600,208.0,/yxHDlr90ww0XZt3U26W95JNExf3.jpg,/g5ZCG8coC1bfB81anxL52l30uDe.jpg,"['Alain Chabat', 'Bae Doona', 'Jules Sagot']",Eric Lartigau,"['seoul, south korea']"
2,177545,455656,#realityhigh,"[{'id': 35, 'name': 'Comedy'}]",When nerdy high schooler Dani finally attracts...,2017-07-17,1.4162,99.0,Released,NaN,6.280,1028.0,/iZliPeiiDta9KbONAhdFSXhTxrO.jpg,/smgZYp49OB6xo4hZewxzryrh5xN.jpg,"['Nesta Cooper', 'Keith Powers', 'Alicia Sanz']",Fernando Lebrija,"['high school', 'nerd', 'teenage crush', 'soci..."
3,117867,252178,'71,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",A young British soldier must find his way back...,2014-10-10,1.3967,99.0,Released,NaN,6.803,1137.0,/xjorsS84euahsmGlnEEeE3LFSVZ.jpg,/aTloiKdNs2c8vlstbx3wBWD6Thi.jpg,"[""Jack O'Connell"", 'Sean Harris', 'Paul Anders...",Yann Demange,"['1970s', 'riot', 'northern ireland', 'surviva..."
4,69757,19913,(500) Days of Summer,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Tom, greeting-card writer and hopeless romanti...",2009-07-17,9.5369,95.0,Released,This is not a love story. This is a story abou...,7.293,10342.0,/qXAuQ9hF30sQRsXf40OfRVl0MJZ.jpg,/1M2i4Mxd03elGOTmEkIvqrHfmyS.jpg,"['Joseph Gordon-Levitt', 'Zooey Deschanel', 'G...",Marc Webb,"['jealousy', 'gallery', 'fight', 'date', 'arch..."


Filter the 80 movies with highest vote count, i.e. the most viewed, so we increase the probability that the user has seen any of these movies.

In [3]:
top_films = df.sort_values(by=['vote_count'], ascending=False)[['movieId', 'id', 'title', 'poster_path']][:80]
top_films.head(10)

,movieId,id,title,poster_path
3683,79132,27205,Inception,/ljsZTbVsrQSqZgWeep2B1QiDKuh.jpg
3742,109487,157336,Interstellar,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg
7427,58559,155,The Dark Knight,/qJ2tW6WMUDux911r6m7haRef0WH.jpg
816,72998,19995,Avatar,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg
2032,122904,293660,Deadpool,/3E53WEZJqP6aM84D8CckXx4pIHw.jpg
7123,89745,24428,The Avengers,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg
821,122912,299536,Avengers: Infinity War,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg
2685,2959,550,Fight Club,/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg
5758,296,680,Pulp Fiction,/vQWk5YBFWF4bZaofAbv0tShwBvQ.jpg
3146,112852,118340,Guardians of the Galaxy,/jPrJPZKJVhvyJ4DmUTrDgmFN0yG.jpg


Now I'll select some movies that I consider are very good films and are pretty poupulars.

In [4]:
my_movies = ["Schindler's List", "La La Land", "Dead Poets Society", "The Silence of the Lambs", "Good Will Hunting", "The Departed", "Spirited Away", 
             "Life Is Beautiful", "City of God", "Cinema Paradiso", "Howl's Moving Castle", "Whiplash", 'Weathering with You', "The Intouchables", "300", 
             "The Maze Runner", "About Time", "I Am Legend", "The Green Mile", "Her"]

In [5]:
mdf = df[df['title'].isin(my_movies)][['movieId', 'id', 'title', 'poster_path']]
mdf.shape

(20, 4)

Verify that the movies that I selected are not in the top films DataFrame

In [6]:
mdf[mdf['id'].isin(top_films['id'])]

,movieId,id,title,poster_path


We don't have duplicated films, so we are going to concatenate both DataFrames

In [7]:
top_films = pd.concat([top_films, mdf])

## Request

In [8]:
api_key = os.getenv('tmdb_api_key')
movies = []
id_errors = []

### Spanish Title

First, I will test the URL to retrieve the title in Spanish and examine how the response is structured.

In [9]:
movie_id = 155 # Inception
url = f"https://api.themoviedb.org/3/movie/{movie_id}/alternative_titles?api_key={api_key}"
response = requests.get(url)
# Verify if the request was successful
if response.status_code == 200:
    # Convert the response to a python dictionary
    data = response.json()
else:
    print(f"Error: {response.status_code}")
    
print(data.keys())

dict_keys(['id', 'titles'])


In [10]:
for title in data['titles']:
    print(title)

{'iso_3166_1': 'US', 'title': 'Batman: The Dark Knight', 'type': ''}
{'iso_3166_1': 'KR', 'title': '배트맨 다크 나이트', 'type': ''}
{'iso_3166_1': 'FR', 'title': 'Batman, Le Chevalier Noir', 'type': ''}
{'iso_3166_1': 'RU', 'title': 'Темный рыцарь', 'type': ''}
{'iso_3166_1': 'PL', 'title': 'Mroczny Rycerz', 'type': ''}
{'iso_3166_1': 'US', 'title': 'Batman II: The Dark Knight', 'type': 'Alternative Title'}
{'iso_3166_1': 'DE', 'title': 'Batman - The Dark Knight', 'type': ''}
{'iso_3166_1': 'US', 'title': 'The Dark Knight 2: The Dark Knight', 'type': 'Full Formal'}
{'iso_3166_1': 'JP', 'title': 'ダークナイト：2008', 'type': ''}
{'iso_3166_1': 'IQ', 'title': 'سوارچاكی تاریكی', 'type': ''}
{'iso_3166_1': 'ES', 'title': 'El caballero oscuro', 'type': 'Español'}
{'iso_3166_1': 'VN', 'title': 'Hiệp Sĩ Bóng Đêm', 'type': ''}
{'iso_3166_1': 'ES', 'title': 'Batman (El caballero oscuro) - 2. El caballero oscuro', 'type': 'Español'}
{'iso_3166_1': 'DE', 'title': 'The Dark Knight 2: The Dark Knight', 'type': '

As we can see, each object in the response is a dictionary with three keys. The most important key is 'iso_3166_1', which represents the country code. To retrieve the title in Spanish, we will focus on the values for 'ES' (Spain) or 'MX' (Mexico).

## Retrieve the Title in Spanish

Once we understand the structure of each response, we will retrieve the spanish title for all the desired movies.

In [11]:
def get_title(movie_id, titles):
    # Initialize the title variables
    mx_title = None
    es_title = None

    # Search for the titles in Spanish (Mexico and Spain)
    for title in titles:
        if title['iso_3166_1'] == 'MX':
            mx_title = title['title']
        elif title['iso_3166_1'] == 'ES':
            es_title = title['title']

    # Return the appropriate title or fallback to English if not found
    if mx_title:
        return mx_title
    elif es_title:
        return es_title

    # If no title found in Spanish, return the English title from the original dataset
    return top_films[top_films['id'] == movie_id]['title'].iat[0]


In [12]:
def get_title_and_poster(ids):
    for movie_id in ids:
        # URLs to get the title in Spanish
        title_url = f"https://api.themoviedb.org/3/movie/{movie_id}/alternative_titles?api_key={api_key}"
        try:
            # Request to fetch the titles
            response = requests.get(title_url)

            # Verify if the request was successful
            if response.status_code == 200:
                titles = response.json()
                # Get the title in Spanish if available, else get it in English
                sp_title = get_title(movie_id, titles['titles'])
                # Append movie details to the movies list
                movies.append([movie_id, sp_title])
            else:
                id_errors.append(movie_id)
        except requests.exceptions.RequestException as e:
            # Handle any network-related or request errors
            print(f"Error fetching data for movie ID {movie_id}: {e}")
            id_errors.append(movie_id)

In [13]:
get_title_and_poster(top_films['id'])
print(f'We have retrieved correctly {len(movies)} and we had {len(id_errors)} problems')

We have retrieved correctly 100 and we had 0 problems


Not lets see how the current titles are

In [14]:
movies[:10]

[[27205, 'El Origen'],
 [157336, 'Interestelar'],
 [155, 'Batman (El caballero oscuro) - 2. El caballero oscuro'],
 [19995, 'Avatar'],
 [293660, 'Deadpool: 1'],
 [24428, 'Vengadores'],
 [299536, 'Vengadores: Infinity War'],
 [550, 'Fight Club'],
 [680, 'Tiempos violentos'],
 [118340, 'Guardianes de la galaxia - Volumen 1']]

In [15]:
# Correct the title of Batman: The Dark Knight
movies[2][1] = 'Batman: El Caballero Oscuro'
movies[:5]

[[27205, 'El Origen'],
 [157336, 'Interestelar'],
 [155, 'Batman: El Caballero Oscuro'],
 [19995, 'Avatar'],
 [293660, 'Deadpool: 1']]

## Add the Poster Path

In [16]:
for idx, poster_path in enumerate(top_films['poster_path']):
    movies[idx].append(poster_path)

In [17]:
movies[:2]

[[27205, 'El Origen', '/ljsZTbVsrQSqZgWeep2B1QiDKuh.jpg'],
 [157336, 'Interestelar', '/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg']]

## Insert them in the DataBase

Once we have all the desired movies with it's title in spnaish and with its poster path, we are going to insert them in the database

In [18]:
# Connect to MySQL
conn = mysql.connector.connect(
    host = os.getenv('DB_HOST'),
    user = os.getenv('DB_USER'),
    password = os.getenv('DB_PASSWORD'),
    database = os.getenv('DB_NAME')
)

cursor = conn.cursor()

In [19]:
for movie in movies:
    cursor.execute("""
        INSERT INTO top_films (movieId, title, poster_path)
        VALUES (%s, %s, %s)
    """, (movie[0], movie[1], movie[2]))

# Commit the changes and Close the connection
conn.commit()
cursor.close()
conn.close()